In [1]:
%pylab
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from metric import tests
import cv2, tqdm
from utils import wand_transforms
import wand
from wand.image import Image
from metric.separate_iou import siou
from sklearn.metrics import pairwise_distances, adjusted_rand_score

In [3]:
#generate random images 100
!mkdir -p expr3_data/compare

In [4]:
for i in range(100):
    M = np.random.randint(25, 40)
    circs = tests.uniform_random_circles(M, 55, 450, 10, 50, 5)
    XY = np.array([c for c, r in circs])
    rs = np.array([r for c, r in circs])
    dist = pairwise_distances(XY)
    dist -= rs[None, :]
    dist -= rs[:, None]
    np.fill_diagonal(dist, 'inf')
    
    N = 1
    nrs = rs.copy()
    for j in range(N):
        c1, c2 = np.unravel_index(dist.argmin(), dist.shape)
        pad = int((dist[c1, c2]/2))+1
        dist[c1, c2] = float('inf')
        dist[c2, c1] = float('inf')
        nrs[c1] = max(nrs[c1], rs[c1]+pad)
        nrs[c2] = max(nrs[c2], rs[c2]+pad)
    
    slices = np.zeros((M, 512, 512), np.uint8)
    image = np.zeros((512, 512), np.uint8)
    for j, (c, r) in enumerate(circs):
        slices[j] = cv2.circle(slices[j], c, r, 1, -1)
        image = cv2.circle(image, c, nrs[j], 255, -1)
    cv2.imwrite(f'./expr3_data/image_{i}.png', image)
    np.save(f'./expr3_data/slices_{i}.npy', slices)
    
    orig = slices.sum(0).astype(uint8)
    tocompare = np.ones((512, 1026), uint8)*255
    tocompare[:, :512] = orig*255
    tocompare[:, 514:] = image
    cv2.imwrite(f'./expr3_data/compare/{i}.png', tocompare)

## Experiment

In [5]:
lst = ['niou', 'ari', 'f0_ari', 'f1_ari', 'ff_ari', 'siou_normal', 'siou_prop', 'siou_inv']
results = {k:[] for k in lst}

for i in tqdm.trange(100):
    slc = np.load(f'./expr3_data/slices_{i}.npy')
    img = cv2.imread(f'./expr3_data/image_{i}.png', cv2.IMREAD_ANYDEPTH)
    img = (img > 127).astype(uint8)
    for normalize in ['normal', 'prop', 'inv']:
        results['siou_'+normalize].append(siou(img, slc, normalize=normalize))
    
    
    orig = slc.sum(0).astype(uint8)
    _, x = cv2.connectedComponents(orig, connectivity=8)
    _, y = cv2.connectedComponents(img, connectivity=8)
    results['ari'].append(adjusted_rand_score(x.reshape(-1), y.reshape(-1)))
    
    idx = orig != 0
    results['f0_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    idx = img != 0
    results['f1_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    idx = (orig != 0) | (y != 0)
    results['ff_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    orig = orig.astype(bool)
    img = img.astype(bool)
    results['niou'].append((orig&img).sum()/(orig|img).sum())
for k in lst:
    ar = np.array(results[k])
    mean, std = ar.mean(), ar.std()
    print(f'{k:15s} {mean:.2f} {std:.2f}')
for k in lst:
    ar = np.array(results[k])
    mean, std = ar.mean(), ar.std()
    print(f'{mean:.2f} $\pm$ {std:.2f}', end=' & ')

100%|██████████| 100/100 [00:28<00:00,  3.51it/s]

niou            0.98 0.01
ari             0.99 0.01
f0_ari          0.97 0.03
f1_ari          0.95 0.05
ff_ari          0.95 0.05
siou_normal     0.96 0.01
siou_prop       0.96 0.02
siou_inv        0.96 0.03
0.98 $\pm$ 0.01 & 0.99 $\pm$ 0.01 & 0.97 $\pm$ 0.03 & 0.95 $\pm$ 0.05 & 0.95 $\pm$ 0.05 & 0.96 $\pm$ 0.01 & 0.96 $\pm$ 0.02 & 0.96 $\pm$ 0.03 & 